# Challenge 1(Parsing HTML)

In [156]:
from scrapy import Item, Field, Selector
#using scrapy Selector function instead of from lxml import html
from datetime import datetime
dateparse = lambda x: datetime.strptime(x, '%d %B %Y')
import pandas as pd
import os
path = os.path.abspath('__file__')
path = path.strip('__file__')
print(path) #to get full path of file
#import requests #requests doesn't support file:// protocol cause its build on HTTP protocl, its useless in our case

/Users/mnabil/workspace/challenges/Heni/Data Engineer Scraping test - 17-06-21/


In [157]:
def local_request(p):
    with open(p,'r') as f:
        body = f.read()
        return body

In [158]:
p = path+"candidateEvalData/webpage.html"
body = local_request(p) 
#making a local request, so we don't use scrapy's downloader middleware and start a crawl process

In [160]:
selector = Selector(text=body) #selecting html code using scrapy Selector class

In [161]:
#implementing my scrapy Item class
class Painting(Item): 
    name = Field()  #name
    artist_name = Field() #artiest name
    price_gbp = Field() #price GBP
    price_us = Field() #price US
    price_gbp_est = Field() #price GBP est
    price_us_est = Field() #price US est
    img_link = Field() #painting link
    sale_date = Field() #sale date

In [162]:
#initializing my item
painting = Painting()

In [163]:
#using parse on my response.Selector , passing item to be filled (Mocking scrapy Mechanism)
def parse_painting(selector, item): #instead of response Object
    painting['name'] = selector.css("h2.itemName").xpath('normalize-space(string())').extract_first()
    painting['artist_name'] = selector.css('.lotName').xpath('text()').re('(.+)?\s\(.+')[0]
    painting['price_gbp'] = selector.css("#main_center_0_lblPriceRealizedPrimary").xpath('text()').re("GBP (.+)?")[0]
    painting['price_us'] = selector.css("#main_center_0_lblPriceRealizedSecondary").xpath('text()').re("USD (.+)?")[0]
    painting['price_gbp_est'] = '-'.join(
        selector.css("#main_center_0_lblPriceEstimatedPrimary").xpath('text()').re("GBP (.+)? - GBP (.+)?")
    )
    painting['price_us_est'] = '-'.join(
        selector.css("#main_center_0_lblPriceEstimatedSecondary").xpath('text()').re("\(USD (.+)? - USD (.+)?\)")
    )
    painting['img_link'] = selector.css('#imgLotImage').xpath('@src').extract_first()
    sale_date = selector.css('#main_center_0_lblSaleDate').xpath('text()').extract_first().strip().strip(',')
    painting['sale_date'] = dateparse(sale_date).strftime("%Y-%m-%d")
    print("here's my Item Dictionary:\n%s\n" %painting) #for logging
    return pd.DataFrame(data = [painting],columns= painting.keys()) #returning pd.dataframe as per request

In [164]:
parse_painting(selector, painting)

here's my Item Dictionary:
{'artist_name': 'Peter Doig',
 'img_link': 'http://www.christies.com/lotfinderimages/D59730/peter_doig_the_architects_home_in_the_ravine_d5973059g.jpg',
 'name': "The Architect's Home in the Ravine",
 'price_gbp': '11,282,500',
 'price_gbp_est': '10,000,000-15,000,000',
 'price_us': '16,370,908',
 'price_us_est': '14,509,999-21,764,999',
 'sale_date': '2016-02-11'}



,name,artist_name,price_gbp,price_us,price_gbp_est,price_us_est,img_link,sale_date
0,The Architect's Home in the Ravine,Peter Doig,"11,282,500","16,370,908","10,000,000-15,000,000","14,509,999-21,764,999",http://www.christies.com/lotfinderimages/D5973...,2016-02-11


# Challenge 2 (Regex)

In [165]:
import pandas as pd
question_df = pd.read_csv("candidateEvalData/dim_df_correct.csv")

In [166]:
question_df

,rawDim,height,width,depth
0,19×52cm,19.0,52.0,NaN
1,"50 x 66,4 cm",50.0,66.4,NaN
2,168.9 x 274.3 x 3.8 cm (66 1/2 x 108 x 1 1/2 in.),168.9,274.3,3.8
3,Sheet: 16 1/4 × 12 1/4 in. (41.3 × 31.1 cm) Im...,35.6,25.1,NaN
4,5 by 5in,12.7,12.7,NaN


In [167]:
#Solution was using named capture groups
print(question_df['rawDim'].str.extract('(?P<height>[\d.]+)\s*(?:[x×]|by)\s*(?P<width>[\d.]+)\s*(?:[x×]\s*(?P<depth>[\d.]+))?\s*(?P<unit>(?:cm|in))').astype({'height': float, 'width': float, 'depth': float}))
question_df['rawDim'][3]
#Issue with regex and conversion

   height  width  depth unit
0    19.0   52.0    NaN   cm
1     NaN    NaN    NaN  NaN
2   168.9  274.3    3.8   cm
3    41.3   31.1    NaN   cm
4     5.0    5.0    NaN   in


'Sheet: 16 1/4 × 12 1/4 in. (41.3 × 31.1 cm) Image: 14 × 9 7/8 in. (35.6 × 25.1 cm)'

In [168]:
#using named capture groups we

In [671]:
regex = r'(?P<height>[\d.]+)\s*(?:[x×]|by)\s*(?P<width>[\d.]+)\s*(?:[x×]\s*(?P<depth>[\d.]+))?\s*(?P<unit>(?:cm|in))'
factors = {'cm': 1, 'in': 2.54}

df2 = (question_df['rawDim'].str.replace(r'(\d+),', r'\1.', regex=True)
         .str.extract(regex)
         .astype({'height': float, 'width': float, 'depth': float})
       )

question_df[['rawDim']].join(df2[['height', 'width', 'depth']].mul(df2['unit'].map(factors), axis=0))

,rawDim,height,width,depth
0,19×52cm,19.0,52.0,NaN
1,"50 x 66,4 cm",50.0,66.4,NaN
2,168.9 x 274.3 x 3.8 cm (66 1/2 x 108 x 1 1/2 in.),168.9,274.3,3.8
3,Sheet: 16 1/4 × 12 1/4 in. (41.3 × 31.1 cm) Im...,41.3,31.1,NaN
4,5 by 5in,12.7,12.7,NaN


In [672]:
#an 30 Mins on this , couldn't get column 3 to match question_df , not even with look_behind regex matching
#one way of solving this i can think of is get the accurate text during scraping then find common pattern
#would love to learn how we solve this using a different solution or regex pattern to get identical DFs

# Challenge 3 (Web Crawler)

#### Scrapy crawler Available in https://github.com/mnabil/Heni_challenge 

In [171]:
#showing scraped Results
bearspace_data = pd.read_csv("bearspace/bear.csv")
bearspace_data.head()

,height_cm,media,price_gbp,title,url,width_cm
0,110.0,"Oil paint, spray paint & gold leaf on board",1600,The Hurricane by Lucy Baker,https://www.bearspace.co.uk/product-page/the-h...,130.0
1,110.0,"Oil paint, spray paint & gold leaf on board",1600,Amazons by Lucy Baker,https://www.bearspace.co.uk/product-page/amazo...,130.0
2,127.0,Wall based sculpture,3400,Candy Flop by Olly Fathers,https://www.bearspace.co.uk/product-page/candy...,53.0
3,120.0,Acrylic paint on sprayed board,4000,Crossroads by Olly Fathers,https://www.bearspace.co.uk/product-page/cross...,120.0
4,30.0,Acrylic paint on sprayed board,240,P10 by Olly Fathers,https://www.bearspace.co.uk/product-page/p10-b...,30.0


# Challenge 4 (Data)

## Task 1 

Inner join: is basically intersection between two tables or two sets

Left join: is selecting left table with matching elements from right table

Right join: is selecting right able with matching elements from left table
(for both cases null is assigned for non-matching elements)

Outer join: is union between both tables

## Task 2

In [172]:
import pandas as pd
flights = pd.read_csv("candidateEvalData/flights.csv")
airports = pd.read_csv("candidateEvalData/airports.csv")
weather = pd.read_csv("candidateEvalData/weather.csv")
airlines = pd.read_csv("candidateEvalData/airlines.csv")

flights.head(3)

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517,515,2.0,830,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400.0,5.0,15.0,2013-01-01T05:00:00Z
1,2013,1,1,533,529,4.0,850,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416.0,5.0,29.0,2013-01-01T05:00:00Z
2,2013,1,1,542,540,2.0,923,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089.0,5.0,40.0,2013-01-01T05:00:00Z


In [173]:
weather.head(3)

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2013.0,1.0,1,0,37.04,21.92,53.97,230.0,10.35702,11.918651,0.0,1013.9,10.0,2013-01-01T00:00:00Z
1,EWR,2013.0,1.0,1,1,37.04,21.92,53.97,230.0,13.80936,15.891535,0.0,1013.0,10.0,2013-01-01T01:00:00Z
2,EWR,2013.0,1.0,1,2,37.94,21.92,52.09,230.0,12.65858,14.567241,0.0,1012.6,10.0,2013-01-01T02:00:00Z


In [174]:
airlines.head(3)


,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.


In [175]:
#Add full airline name to the flights dataframe and show the arr_time, origin, dest and the name of the airline.

In [176]:
import pandasql

In [177]:
flights_airline_query = "SELECT flights.arr_time, flights.origin, flights.dest, airlines.name FROM flights JOIN airlines ON flights.carrier = airlines.carrier;"

In [178]:
fligths_airline_df = pandasql.sqldf(flights_airline_query, globals())

In [179]:
fligths_airline_df

,arr_time,origin,dest,name
0,830,EWR,IAH,United Air Lines Inc.
1,850,LGA,IAH,United Air Lines Inc.
2,923,JFK,MIA,American Airlines Inc.
3,1004,JFK,BQN,JetBlue Airways
4,812,LGA,ATL,Delta Air Lines Inc.
5,740,EWR,ORD,United Air Lines Inc.
6,913,EWR,FLL,JetBlue Airways
7,709,LGA,IAD,ExpressJet Airlines Inc.
8,838,JFK,MCO,JetBlue Airways
9,753,LGA,ORD,American Airlines Inc.


In [180]:
jet_blue_df = fligths_airline_df[fligths_airline_df.name.str.contains("JetBlue")]
jet_blue_df

,arr_time,origin,dest,name
3,1004,JFK,BQN,JetBlue Airways
6,913,EWR,FLL,JetBlue Airways
8,838,JFK,MCO,JetBlue Airways
10,849,JFK,PBI,JetBlue Airways
11,853,JFK,TPA,JetBlue Airways
15,702,JFK,BOS,JetBlue Airways
17,851,LGA,FLL,JetBlue Airways
19,844,EWR,PBI,JetBlue Airways


In [181]:
#Summarise the total number of flights by origin in ascending.
fligths_airline_df["numFlights"] = fligths_airline_df.groupby("origin").cumcount()+1

In [184]:
#Filter resulting data.frame to return only origins with more than 10,000 flights. ??? no such count at 10k+ ?
filterd_flights_df = fligths_airline_df[fligths_airline_df.numFlights > 6]

In [185]:
filterd_flights_df

,arr_time,origin,dest,name,numFlights
15,702,JFK,BOS,JetBlue Airways,7
18,837,LGA,ATL,Envoy Air,7
